#HW1 - Data Exploration and Preparation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Part 1: Data Loading and First Look

## (Q1) Loading the dataset

In [2]:
dataset = pd.read_csv("virus_data.csv")
dataset.shape

(1250, 25)

## (Q2)

In [ ]:
dataset['conversations_per_day'].value_counts()

## (Q3)

In [ ]:
dataset.dtypes

## (Q4)

In [3]:
random_state = 17 + 75
train, test = train_test_split(dataset, test_size=0.2, random_state=random_state)
train.shape, test.shape

((1000, 25), (250, 25))

# Part 2: Missing Values and Outliers

## (Q5)

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

## (Q6)

In [8]:
def plot_box(series, title, xlabel):
    cleaned_series = series.dropna()
    plt.figure(figsize=(8, 6))
    plt.boxplot(cleaned_series,
    vert=False)
    plt.xlabel(xlabel)
    plt.title(title)

In [ ]:
plot_box(train['PCR_02'], 'PCR_02 BOX PLOT', 'PCR_02')

In [ ]:
plot_box(train['household_income'], 'household income BOX PLOT', 'Household Income')

In [ ]:
print(train['household_income'].describe())
print(train['PCR_02'].describe())

## (Q7)

In [ ]:
## fill missing values with median
filled_train_median = train.fillna({'household_income':train.household_income.median(), 'PCR_02':train.PCR_02.median()})
filled_test_median = test.fillna({'household_income':train.household_income.median(), 'PCR_02':train.PCR_02.median()})

print(filled_train_median['household_income'].describe())
print(filled_train_median['PCR_02'].describe())

# Part 3: Warming up with k-Nearest Neighbors

# Basic data exploration

## (Q8)

In [ ]:
g=sns.pairplot(filled_train_median,
               vars=['PCR_01' , 'PCR_02' , 'PCR_08', 'PCR_10'],
               hue="spread",
               plot_kws={"s": 12})
g.fig.suptitle("PCR features pairplot", y=1.04)
for ax in np.ravel(g.axes):
    ax.grid(alpha=0.5)

g.fig.set_size_inches(12,8)

In [ ]:
g=sns.pairplot(filled_train_median,
               x_vars=['PCR_01'],
               y_vars=['PCR_08'],
               hue="spread",
               plot_kws={"s": 12})
g.fig.suptitle("PCR_01 PCR_08 pairplot", y=1.04)
for ax in np.ravel(g.axes):
    ax.grid(alpha=0.5)

g.fig.set_size_inches(12,8)

## (Q9)

In [ ]:
print("Correlation between PCR_01 and spread is: {:.4f}".format(filled_train_median['PCR_01'].corr(filled_train_median['spread'])))
print("Correlation between PCR_08 and spread is: {:.4f}".format(filled_train_median['PCR_08'].corr(filled_train_median['spread'])))

# k-NN implementation

## Task D

In [ ]:
import visualize_clf
import kNN

# temporal data frames for PCR_01 and PCR_08
temp_df = filled_train_median[['PCR_01','PCR_08']]
temp_test_df = filled_test_median[['PCR_01','PCR_08']]

# kNN model
model = kNN.kNN(n_neighbors=1)
model.fit(temp_df,filled_train_median['spread'])
visualize_clf.visualize_clf(model, temp_df, filled_train_median['spread'], "Decision Regions", "PCR_01", "PCR_08")

# train and test accuracy
print("Train accuracy is: {:.3f}".format(model.score(temp_df, filled_train_median['spread'])))
print("Test accuracy is: {:.3f}".format(model.score(temp_test_df, filled_test_median['spread'])))

# Data Normalization

## (Q12)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# normalize the data
scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(temp_df)
normed_temp_df = scaler.transform(temp_df)
normed_temp_test_df = scaler.transform(temp_test_df)

# kNN model k=1
model = kNN.kNN(n_neighbors=1)
model.fit(normed_temp_df, filled_train_median['spread'])
visualize_clf.visualize_clf(model, normed_temp_df, filled_train_median['spread'], "Decision Regions", "PCR_01", "PCR_08")

# train and test accuracy
print("Train accuracy is: {:.3f}".format(model.score(normed_temp_df, filled_train_median['spread'])))
print("Test accuracy is: {:.3f}".format(model.score(normed_temp_test_df, filled_test_median['spread'])))


## (Q13)

In [ ]:
# kNN model k=9
model = kNN.kNN(n_neighbors=9)
model.fit(normed_temp_df, filled_train_median['spread'])
visualize_clf.visualize_clf(model, normed_temp_df, filled_train_median['spread'], "Decision Regions", "PCR_01", "PCR_08")
# train and test accuracy
print("Train accuracy is: {:.3f}".format(model.score(normed_temp_df, filled_train_median['spread'])))
print("Test accuracy is: {:.3f}".format(model.score(normed_temp_test_df, filled_test_median['spread'])))

# Part 4: Data Exploration

# Task D

In [19]:
# define special property feature
df_bool= pd.DataFrame({'SpecialProperty' : filled_train_median["blood_type"].isin(["O+", "B+"])})
filled_train_median_bool = pd.concat([filled_train_median, df_bool], axis=1)
filled_train_median_bool.drop(columns=['blood_type'], inplace=True)

# Univariate Analysis

In [ ]:
COL_NAME = ['PCR_01', 'PCR_02', 'PCR_03', 'PCR_04', 'PCR_05', 'PCR_06', 'PCR_07', 'PCR_08', 'PCR_09', 'PCR_10', 'weight', 'household_income']
COLS, ROWS = (2, len(COL_NAME))
plt.figure(figsize=(5 * COLS, 4 * ROWS))
for row in range(ROWS):
    column = COL_NAME[row]
    for j, cls in enumerate(["risk", "spread"]):
        plt.subplot(ROWS,COLS, row * COLS + 1 + j)
        isContinuous = "float" in filled_train_median_bool[column].dtype.name
        sns.histplot(data=filled_train_median_bool, x=column, hue=cls, line_kws={"linewidth": 3},
        kde=isContinuous, multiple="layer" if isContinuous else "dodge")
        plt.grid(alpha=0.5)
plt.tight_layout()

# (Q15)

In [ ]:
plt.figure()
sns.histplot(data=filled_train_median_bool, x="weight", hue="spread", line_kws={"linewidth": 3},
kde=True, multiple="layer")
plt.grid(alpha=0.5)
plt.tight_layout()

# (Q16)

In [ ]:
plt.figure()
sns.histplot(data=filled_train_median_bool, x="PCR_04", hue="risk", line_kws={"linewidth": 3},
kde=True, multiple="layer")
plt.grid(alpha=0.5)
plt.tight_layout()

# Bivariate Analysis

# (Q17)

In [23]:
# split the data by the special property feature
filled_train_median_true = filled_train_median_bool.groupby('SpecialProperty').get_group(True)
filled_train_median_false = filled_train_median_bool.groupby('SpecialProperty').get_group(False)

In [ ]:
g=sns.pairplot(filled_train_median_true,
               vars=['PCR_04' , 'PCR_05' , 'PCR_06'],
               hue="risk",
               plot_kws={"s": 12})
g.fig.suptitle("PCR features pairplot for blood_type in {O+, B+}", y=1.04)
for ax in np.ravel(g.axes):
    ax.grid(alpha=0.5)

g.fig.set_size_inches(12,8)

In [ ]:
g=sns.pairplot(filled_train_median_false,
               vars=['PCR_04' , 'PCR_05' , 'PCR_06'],
               hue="risk",
               plot_kws={"s": 12})
g.fig.suptitle("PCR features pairplot for blood_type not in {O+, B+}", y=1.04)
for ax in np.ravel(g.axes):
    ax.grid(alpha=0.5)

g.fig.set_size_inches(12,8)

# (Q18)

In [ ]:
g = sns.jointplot(data=filled_train_median_true, x="PCR_04", y="PCR_06", hue="risk")
g.fig.suptitle("PCR_04 vs. PCR_06 for blood_type in {O+, B+}", y=1.04)
_ = g.ax_joint.grid(alpha=0.5)

In [ ]:
g = sns.jointplot(data=filled_train_median_false, x="PCR_04", y="PCR_06", hue="risk")
g.fig.suptitle("PCR_04 vs. PCR_06 for blood_type not in {O+, B+}", y=1.04)
_ = g.ax_joint.grid(alpha=0.5)

In [ ]:
g = sns.jointplot(data=filled_train_median_bool, x="PCR_04", y="PCR_06", hue="risk")
g.fig.suptitle("PCR_04 vs. PCR_06 for all blood_type", y=1.04)
_ = g.ax_joint.grid(alpha=0.5)

# The risk target feature

# (Q19)

In [29]:
def plot3d(df, colX, colY, colZ, title, hue=None, s=1):
  # %matplotlib inline

  fig = plt.figure(figsize=(20, 4))
  axes = [fig.add_subplot(1, 4, 1, projection='3d'),
          fig.add_subplot(1, 4, 2, projection='3d'),
          fig.add_subplot(1, 4, 3, projection='3d'),
          fig.add_subplot(1, 4, 4, projection='3d')]

  palette = sns.color_palette("hls", 2)

  for i,ax in enumerate(axes):
    if hue is None:
      ax.scatter3D(df[colX], df[colY], df[colZ], color=palette[0], s=s)
    else:
      idx = df[hue] == True
      df1 = df[idx]
      df2 = df[~idx]

      ax.scatter3D(df1[colX], df1[colY], df1[colZ], color=palette[0], s=s)
      ax.scatter3D(df2[colX], df2[colY], df2[colZ], color=palette[1], s=s)

    if i == 3 or i == 1:
      ax.set_xlabel(colX, labelpad=20)
      ax.set_ylabel(colY, labelpad=20)
    else:
      ax.xaxis.set_ticklabels([])
      ax.yaxis.set_ticklabels([])

    ax.view_init(elev=5., azim=45*i+15)

  axes[1].set_zlabel("\n" + colZ, labelpad=1)

  plt.suptitle(title)
  plt.subplots_adjust(wspace=0.1, top=0.99)
  plt.show()

In [ ]:
plot3d(filled_train_median_bool, "PCR_04", "SpecialProperty", "PCR_06", title="PCR_04, PCR_06, SpecialProperty colored by risk value", hue="risk", s=5)

## Part 5: More Data Normalization

In [31]:
from sklearn.preprocessing import StandardScaler

# scaler for minmax and standart
scaler_standart = StandardScaler()
scaler_minmax = MinMaxScaler(feature_range=(-1,1))

# temporal data frames for minmax and standart
temp_df_minmax = filled_train_median_bool[['PCR_01','PCR_03','PCR_04','PCR_06','PCR_08']]
temp_df_standart = filled_train_median_bool[['PCR_02','PCR_05','PCR_07','PCR_09','PCR_10']]
normed_filled_train_median_bool = filled_train_median_bool.copy()

# normalize the data
scaler_minmax.fit(temp_df_minmax)
scaler_standart.fit(temp_df_standart)
normed_filled_train_median_bool[['PCR_01','PCR_03','PCR_04','PCR_06','PCR_08']] = scaler_minmax.transform(temp_df_minmax)
normed_filled_train_median_bool[['PCR_02','PCR_05','PCR_07','PCR_09','PCR_10']] = scaler_standart.transform(temp_df_standart)

## Part 6: Data Preparation Pipeline

In [7]:
from prepare import prepare_data

# Prepare training set according to itself
train_df_prepared = prepare_data(train, train)
# Prepare test set according to the raw training set
test_df_prepared = prepare_data(train, test)

# test_df_prepared.to_csv("test_df_prepared.csv", index=False)

(1000, 25)
(1000, 25)
(1000, 26)
(1000, 25)
(250, 25)
(250, 25)
(250, 26)
(250, 25)
